# Head

In [72]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE/InitMamba")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%pip install --no-build-isolation --verbose ./InitMamba

In [ ]:
!python setup.py bdist_wheel

In [ ]:
!unzip dist/mamba_ssm-1.0-py3-none-any.whl -d ./wheel_content

In [49]:
!dir /usr/local/lib/python3.11/dist-packages/InitMamba/

Block.py     layer_norm.py  __pycache__			 softplus.py
CausalLM.py  Model.py	    selective_scan_interface.py  torch_utils.py
__init__.py  Output.py	    selective_state_update.py


In [63]:
import InitMamba

In [61]:
import torch, transformers, importlib
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

In [64]:
import selective_scan_cuda
selective_scan_cuda.__file__

'/usr/local/lib/python3.11/dist-packages/selective_scan_cuda.cpython-311-x86_64-linux-gnu.so'

In [24]:
import selective_scan_cuda
selective_scan_cuda.__file__

'/usr/local/lib/python3.11/dist-packages/selective_scan_cuda.cpython-311-x86_64-linux-gnu.so'

# Lab

In [65]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
inputs = tokenizer(["Today is Monday, Tomorrow is Tuesday.",
           "Today is Sunday, Tomorrow is Monday.",
           "Today is Sunday, Tomorrow is Monday."],
            max_length = 32,
           padding = 'max_length',
           return_tensors='pt').to('cuda')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [71]:
importlib.reload(InitMamba)
import InitMamba

model1 = InitMamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").cuda()
# model1.train()

res1 = model1(**inputs, output_ssm_last_states = True)
ssm_last_states = res1.ssm_last_states
print(ssm_last_states[-1])

torch.float32


In [77]:
model1.eval()
res = model1(**inputs, output_ssm_last_states = True, inputs_ssm_states = ssm_last_states)
tokenizer.batch_decode(res.logits.argmax(-1))

[' is Wednesday, Tomorrow is Tuesday,\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>',
 ' is Tuesday, Tomorrow is Monday,\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>',
 ' is Tuesday, Tomorrow is Monday,\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|

In [75]:
res.logits

tensor([[[-50.6909, -66.3721, -47.7312,  ..., -66.4962, -66.2791, -66.5470],
         [-51.3856, -67.4400, -50.2360,  ..., -67.6238, -67.5241, -67.5715],
         [-30.5410, -48.7618, -29.7472,  ..., -48.6926, -48.7305, -48.9154],
         ...,
         [273.4377, -29.3720,  61.1656,  ..., -29.2746, -29.3745, -29.4359],
         [273.4377, -29.3720,  61.1656,  ..., -29.2746, -29.3745, -29.4359],
         [273.4377, -29.3720,  61.1656,  ..., -29.2746, -29.3745, -29.4359]],

        [[-50.5575, -66.2595, -47.6861,  ..., -66.3802, -66.1592, -66.4359],
         [-41.8695, -58.5675, -40.6946,  ..., -58.7701, -58.6721, -58.6985],
         [-39.3041, -57.5434, -38.2460,  ..., -57.5422, -57.4608, -57.6744],
         ...,
         [273.4377, -29.3720,  61.1656,  ..., -29.2746, -29.3745, -29.4359],
         [273.4377, -29.3720,  61.1656,  ..., -29.2746, -29.3745, -29.4359],
         [273.4377, -29.3720,  61.1656,  ..., -29.2746, -29.3745, -29.4359]],

        [[-50.5575, -66.2595, -47.6861,  ...

In [69]:
torch.autograd.grad(outputs=res.logits.mean(), inputs=ssm_last_states[0])

(tensor([[[ 0.0000e+00, -4.1876e-05, -4.3776e-03,  ..., -7.8745e-16,
           -7.7959e-09,  5.1060e-17],
          [ 1.0397e-02, -3.0742e-07, -6.2853e-05,  ..., -6.4466e-06,
           -0.0000e+00,  5.4878e-03],
          [-8.0023e-13,  1.0602e-03,  1.0364e-02,  ...,  4.8393e-25,
            3.7654e-13, -0.0000e+00],
          ...,
          [-1.0494e-05,  3.2375e-19,  7.0474e-04,  ..., -8.3368e-03,
           -3.1304e-09,  3.9292e-11],
          [ 1.0459e-02, -1.0855e-02,  1.0058e-33,  ..., -1.4776e-05,
            0.0000e+00, -0.0000e+00],
          [ 2.6720e-03, -2.4888e-05, -1.8886e-03,  ..., -3.2337e-05,
           -0.0000e+00,  1.4410e-03]],
 
         [[ 0.0000e+00, -4.1802e-05,  1.8784e-03,  ..., -8.0543e-16,
           -7.9648e-09,  5.2227e-17],
          [ 5.0486e-03, -1.3516e-07, -2.7047e-05,  ..., -2.8353e-06,
           -0.0000e+00,  3.1578e-03],
          [-8.0727e-13,  9.9191e-04,  8.8240e-03,  ...,  4.8818e-25,
            3.7984e-13, -0.0000e+00],
          ...,
    

In [70]:
# Model without initial SSM states gives random results
print(tokenizer.batch_decode(res1.logits.argmax(dim=-1)))

# Model with initial SSM states gives "Sunday"
res2 = model1(input_ids, inputs_ssm_states = ssm_last_states)
print(tokenizer.batch_decode(res2.logits.argmax(dim=-1)))

[', the, and is Tuesday,\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>', ', the, and is Monday,\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>', ', the, and is Monday,\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftex

NameError: name 'input_ids' is not defined

# Check same

In [ ]:
model2 = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config)

model1.eval()
model2.eval()

print((model2(input_ids).logits - model1(input_ids).logits).abs().sum())